# 利用逻辑回归机器学习预测血糖值

在采集的血糖数据集基础上，进行机器学习，并预测血糖是否超标 （>6.1）

前期血糖采集原理：采用透射式近红外光谱法，嵌入式微处理器分别驱动红光和红外光发射端照射人体指尖部位，光电接收端负责收集透射光，再通过信息处理模块进行信号的放大滤波和光电转换，通过朗伯比尔定律计算得到红光与红外光透射能量的比例值与初始血糖值；再通过能量代谢守恒法进行血糖测量值的修正，修正参数包括人体血氧饱和度、人体心率值、手指指尖体表温度值、环境温度值及体辐射能量值。由于还原性血红蛋白对红光的吸收强，但对红外光的吸收相对较弱；血红蛋白并带有氧分子的血红细胞对红光的吸收比较弱，对红外光的吸收比较强，因此原始光电容积脉搏波信号在上位机进行处理后，通过检测二者的差异，就能得到在不同波段下的光吸收差异，进而就能得到红光与红外光的透射能量比值，将这个比值进行最小二乘建模能得到血氧饱和度值，同时再通过数字温度传感器获得所处的环境问题、被测部位体表温度以及辐射能量；讲这些数据计算完并以表格（.CSV）形式保存，再结合购买的有创血糖仪所测得的标准血糖浓度一起进行数据分析（机器学习）。



In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

计划利用kaggle平台提供的python库、机器学习算法库（逻辑回归算法）以及数据图形显示库，对采集到数据集进行机器学习与预测，从而实现对血糖的持续无创监测。
 （1）利用panda来读取采集到的数据（csv格式）；
 （2）利用sklearn中的逻辑回归算法进行数据分析；
 （3）利用seaborn和matplotlib进行分析结果的可视化展示；
 （4）利用joblib用于存储模型，用于预测用。

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
from sklearn.linear_model import LogisticRegression
import joblib


In [ ]:
diabetesDF = pd.read_csv('../input/hmdata2019-lyh/hm_data_2019_lyh.csv')
print(diabetesDF.head())

数据检查（如没有空值）

In [ ]:
diabetesDF.info() # output shown below

首先查找每一对特征（和结果变量）的相关性，并使用热图可视化相关性.

In [ ]:
corr = diabetesDF.corr()
print(corr)
sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns)

#sns.pairplot(corr, vars = ['Glucose', 'BloodPressure'])


在上面的热图中，更亮的颜色表示相关性更强。从表格和热图可以看出，人体血氧饱和度、人体心率值、手指指尖体表温度值、环境温度值及体辐射能量值都与血糖高低有显著相关性。还要注意两个特征之间的相关性，比如血氧饱和度和心跳，或者血压和体表温度。

****# 数据集准备 (拆分与正规化)**
在使用机器学习算法时，我们应该把数据分成训练集和测试集。（如果我们正在运行的实验数量很大，那么我们应该将数据分为3部分，即训练集、开发集和测试集）。在本例中，我们还将分离出一些数据进行手动交叉检查。
数据集包括总共297项记录。为了训练我们的模型，我们将使用200条记录。我们将使用82条记录进行测试，最后15条记录交叉检查我们的模型。

In [ ]:
dfTrain = diabetesDF[:200]
dfTest = diabetesDF[200:282]
dfCheck = diabetesDF[282:15]

接下来，我们对标签和特征列进行拆分与定义（对于训练和测试数据集）。除此之外，我们还将把它们转换成NumPy数组，因为我们的机器学习算法处理NumPy数组格式的数据。

In [ ]:
trainLabel = np.asarray(dfTrain['DiabetesPredicted'])
trainData = np.asarray(dfTrain.drop(['Date','Time','DiabetesPredicted'],1))
testLabel = np.asarray(dfTest['DiabetesPredicted'])
testData = np.asarray(dfTest.drop(['Date','Time','DiabetesPredicted'],1))

作为使用机器学习之前的最后一步，我们将规范化我们的输入。机器学习模型通常受益于输入规范化。这也使我们更容易理解每个特征的重要性，稍后我们将在查看模型权重时。我们将标准化数据，使每个变量的平均值为0，标准偏差为1

In [ ]:
means = np.mean(trainData, axis=0)
stds = np.std(trainData, axis=0)
trainData = (trainData - means)/stds
testData = (testData - means)/stds
# np.mean(trainData, axis=0) => check that new means equal 0
# np.std(trainData, axis=0) => check that new stds equal 1

# 训练和评估机器学习模型
我们现在可以训练我们的分类模型。我们将使用一个叫做逻辑回归的机器简单学习模型。由于该模型在sklearn中很容易获得，所以训练过程非常简单，我们可以用几行代码来完成。首先，我们创建一个名为diabetesCheck的实例，然后使用fit函数来训练模型。

In [ ]:
diabetesCheck = LogisticRegression()
diabetesCheck.fit(trainData, trainLabel)



接下来，我们将使用我们的测试数据来找出模型的准确性。

In [ ]:
accuracy = diabetesCheck.score(testData, testLabel)
print("accuracy = ", accuracy * 100, "%")

# 解释机器学习模型


为了更好地了解logistic回归模型内部的情况，我们可以可视化我们的模型如何使用不同的特征以及哪些特征具有更大的影响。

In [ ]:
print(corr.columns[1:9])

In [ ]:
coeff = list(diabetesCheck.coef_[0])
labels = list (corr.columns[1:10])

features = pd.DataFrame()
features['Features'] = labels
features['importance'] = coeff
features.sort_values(by=['importance'], ascending=True, inplace=True)
features['positive'] = features['importance'] > 0
features.set_index('Features', inplace=True)
features.importance.plot(kind='barh', figsize=(11, 6),color = features.positive.map({True: 'blue', False: 'red'}))
plt.xlabel('Importance')

# 保存模型
将训练后的模型保存起来用于后续血糖预测.

In [ ]:
joblib.dump([diabetesCheck, means, stds], 'diabeteseModel.pkl')

为了检查我们是否正确地保存了模型，我们将使用我们的测试数据来检查我们保存的模型的准确性（如果我们正确地保存了模型，我们应该观察到准确性没有变化）。

In [ ]:
diabetesLoadedModel, means, stds = joblib.load('diabeteseModel.pkl')
accuracyModel = diabetesLoadedModel.score(testData, testLabel)
print("accuracy = ",accuracyModel * 100,"%")

# 利用模型进行预测

现在，我们将使用未使用的数据来了解如何进行预测。首先检查未使用的数据.

In [ ]:
print(dfCheck.head())

用第一条数据记录来做预测

In [ ]:
sampleData = dfCheck[:1]
# prepare sample
sampleDataFeatures = np.asarray(sampleData.drop(['Date','Time','DiabetesPredicted'],1))
sampleDataFeatures = (sampleDataFeatures - means)/stds
# predict
predictionProbability = diabetesLoadedModel.predict_proba(sampleDataFeatures)
prediction = diabetesLoadedModel.predict(sampleDataFeatures)
print('Probability:', predictionProbability)
print('prediction:', prediction)

# 接下来

 限于时间因素，这里采集的血糖为每天1次，并且自动检测的血糖数据过少，没有将人体辐射能力计算进来，尤其是空腹、餐后、运动后等特定事件下的数据收集还不完整，需要对这些值进一步收集完善，来优化模型的准确度。

